In [21]:
# Replace with your MongoDB connection details
import pymongo
import requests
from PIL import Image
import cv2
import numpy as np
import face_recognition
from facenet_pytorch import MTCNN, InceptionResnetV1
import os
import pickle
from io import BytesIO
from PIL import Image
import requests
import re
from deepface import DeepFace
from facenet_pytorch import MTCNN, InceptionResnetV1


### CONNECT_&_GET_FILES_FROM_DATABASE

In [22]:
def connect_to_db(database_name="officialDB",collection_name="officialDB"):
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client[database_name]
    collection = db[collection_name]
    return collection


def get_shareable_link_from_google_drive(url):
    file_id_match = re.search(r'[-\w]{25,}', url)
    direct_link=""
    if file_id_match:
        file_id = file_id_match.group(0)
        # Construct the direct download link
        direct_link = f"https://drive.google.com/uc?export=view&id={file_id}"
    return direct_link

def get_fields_items(collection):
    name_list = []
    link_list = []
    cursor = collection.find()  # Get all documents in the collection
    for document in cursor:
        name_list.append(document["ID"])
        new_link = get_shareable_link_from_google_drive(document["URL"])
        link_list.append(new_link)
    return name_list, link_list
collection=connect_to_db()
name_list,link_list = get_fields_items(collection)

### GET_IMAGE_&_CREATE_EMBEDDING

In [23]:

def get_image(image_url,model_type="face_recognition"):
    response = requests.get(image_url)
    if response.status_code==200:

    # response.raise_for_status()  # Raise an error if download fails

    # Load the image using PIL for error handling
        img = Image.open(BytesIO(response.content))
        # cv2.imshow("image", np.array(img))
        image = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
        if model_type == "face_recognition":
            face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=2, model="hog")
            if len(face_locations) == 0:
                print(f"No faces detected in image: {image_url}")
                return None

            face_encoding = face_recognition.face_encodings(image, face_locations)[0]  # Handle single face only
            return face_encoding
        if model_type=="facenet":
            mtcnn = MTCNN(image_size=160, margin=0, min_face_size=20)
            resnet = InceptionResnetV1(pretrained='vggface2').eval()
            img_cropped = mtcnn(image)
            if img_cropped is not None:
                img_embedding = resnet(img_cropped.unsqueeze(0))
                # print(len(img_embedding))
                # print(len(img_embedding))
                img_embedding = img_embedding.detach().numpy()
                # print(type(img_embedding))
                return img_embedding
            else:
                print(f"No faces detected in image: {image_url}")
        if model_type=="deepface":
                try:
                    embedding=DeepFace.represent(image)
                    print(type(embedding))
                    if len(embedding):
                        return embedding
                except:
                    print(f"No faces detected in image: {image_url}")
    return None



def get_embedding_and_name_list(name_list,link_list,model_type="face_recognition"):


    known_face_encodings = []
    known_face_names = []
    for   name, link  in zip(name_list, link_list):
        face_encoding = get_image(link,model_type)
        if face_encoding is not None:
            known_face_encodings.append(face_encoding)
            known_face_names.append(name)
    return known_face_encodings , known_face_names




### FACE-RECOGNITION

In [24]:

model_type="face_recognition"
known_face_encodings , known_face_names=get_embedding_and_name_list(name_list,link_list,model_type)

# # print(known_face_encodings)
# # print(known_face_names)
with open(f'{model_type}_known_faces.pickle', 'wb') as f:
    pickle.dump((known_face_encodings, known_face_names), f)

print(len(known_face_encodings))
print(len(known_face_names))

No faces detected in image: https://drive.google.com/uc?export=view&id=1GK-rdA0MZVs_jx1YkWUdFuRFjG24Cp3I
No faces detected in image: https://drive.google.com/uc?export=view&id=1c2dfJ51Ytlemp6eF-ogjiIKTL1hrnvqa
No faces detected in image: https://drive.google.com/uc?export=view&id=1C6WgNu8zH85rqTzCMRFyhbCb4QuSE3bK
No faces detected in image: https://drive.google.com/uc?export=view&id=1duYnUTrj4uAmutYsAF_5wWIk1fxIKaj0
21
21


### FACE-NET

In [28]:

model_type="facenet"
known_face_names = []
known_face_encodings = []

known_face_encodings , known_face_names=get_embedding_and_name_list(name_list,link_list,model_type)

# # print(known_face_encodings)
# # print(known_face_names)
with open(f'{model_type}_known_faces.pickle', 'wb') as f:
    pickle.dump((known_face_encodings, known_face_names), f)

print(len(known_face_encodings))
print(len(known_face_names))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
25
25


### DEEP-FACE

In [16]:
###
model_type="deepface"
known_face_names = []
known_face_encodings = []

known_face_encodings , known_face_names=get_embedding_and_name_list(name_list,link_list,model_type)

# # print(known_face_encodings)
# # print(known_face_names)
with open(f'{model_type}_known_faces.pickle', 'wb') as f:
    pickle.dump((known_face_encodings, known_face_names), f)

print(len(known_face_encodings))
print(len(known_face_names))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
No faces detected in image: https://drive.google.com/uc?export=view&id=1PvpNIF1HuxZu3Aw7m7x8UcmznnF2NzlX
No faces detected in image: https://drive.google.com/uc?export=view&id=1hwzTDouxCvQV9_vuvUJpDFie2KPINRt8
No faces detected in image: https://drive.google.com/uc?export=view&id=1GK-rdA0MZVs_jx1YkWUdFuRFjG24Cp3I
<class 'list'>
No faces detected in image: https://drive.google.com/uc?export=view&id=1lE6Zl750VkykzvTJQ_KxkVobC0RurrgN
<class 'list'>
No faces detected in image: https://drive.google.com/uc?export=view&id=1c2dfJ51Ytlemp6eF-ogjiIKTL1hrnvqa
<class 'list'>
<class 'list'>
<class 'list'>
No faces detected in image: https://drive.google.com/uc?export=view&id=1C6WgNu8zH85rqTzCMRFyhbCb4QuSE3bK
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
No faces detected in image: https://drive.google.com/uc?export=view&id=1duYnUTrj4uAmutYsAF_5wWIk1fxIKaj0
<class 'list'>
No faces detected in imag

### UPDATE EMBEDDING 

In [ ]:
def get_updated_embedding_and_name_list(name_list, link_list, id_list, model_type):
    known_face_encodings = []
    known_face_names = []

    with open(f'{model_type}_known_faces.pickle', 'rb') as f:
        known_face_encodings, known_face_names = pickle.load(f)

    for name, link in zip(name_list, link_list):
        if name in known_face_names:
            continue

        face_encoding = get_image(link)
        if face_encoding is not None:
            known_face_encodings.append(face_encoding)
            known_face_names.append(name)

    with open(f'{model_type}_known_faces.pickle', 'wb') as f:
        pickle.dump((known_face_encodings, known_face_names), f)

    # return known_face_encodings, known_face_names, ids
model_type="face_recognition"
get_updated_embedding_and_name_list(name_list,link_list,model_type)

No faces detected in image: https://drive.google.com/uc?export=view&id=1GK-rdA0MZVs_jx1YkWUdFuRFjG24Cp3I
No faces detected in image: https://drive.google.com/uc?export=view&id=1c2dfJ51Ytlemp6eF-ogjiIKTL1hrnvqa
No faces detected in image: https://drive.google.com/uc?export=view&id=1C6WgNu8zH85rqTzCMRFyhbCb4QuSE3bK
No faces detected in image: https://drive.google.com/uc?export=view&id=1duYnUTrj4uAmutYsAF_5wWIk1fxIKaj0


In [32]:
import pickle

model_type = "facenet"
known=[]
names=[]
ids=[]
with open(f'{model_type}_known_faces.pickle', 'rb') as f:
    known, names = pickle.load(f)

print("Length of name list:", len(known))
print("Length of face embeddings:", len(names))
for i in known:
    print(type(i))

Length of name list: 25
Length of face embeddings: 25
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
